In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys

import pandas as pd
import numpy as np

import time

import torch.nn.functional as F
import torch

import copy

from tqdm.auto import tqdm

In [2]:
from datasets import load_original_dataset, load_deleted_dataset
from models import CNN

In [3]:
DATA_DIR = 'Datasets/Features/'
BATCH_SIZE = 32
EPOCHS = 5
PERCENTAGES = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [4]:
os.makedirs('results/unrolling-sgd', exist_ok=True)

# finetune

In [14]:
def fit(model, save_dir, train_set, test_set, forget_set):
    
    os.makedirs(save_dir, exist_ok=True)
    
    # prepare model
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L94
    M = copy.deepcopy(model)
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L150
    M_unlearned = copy.deepcopy(model)
        
    M_unlearned.train()
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L131
    error = torch.nn.CrossEntropyLoss()
    
    forget_loader = torch.utils.data.DataLoader(forget_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    train_times = list()
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):    
        
        # train
        
        train_time = 0
        
        start_time = time.time()
        
        for x, y in forget_loader:

            output = M_unlearned(x.cuda())
            y = y.cuda()
            loss = error(output, y)
            
            # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L163
            loss.backward(retain_graph=True)
            grads = torch.autograd.grad(loss, [param for param in M_unlearned.parameters()], create_graph = True)
            
            old_params = {}
            for i, (name, params) in enumerate(M.named_parameters()):
                    old_params[name] = params.clone()
                    old_params[name] += LR * grads[i]
            for name, params in M_unlearned.named_parameters():
                    params.data.copy_(old_params[name])
            
            train_time += time.time() - start_time
            
            start_time = time.time()
            
        train_times.append(train_time)
        
        # test
            
        M_unlearned.eval()
        with torch.no_grad():
            
            #
            
            x, y = train_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            train_accs.append(np.mean(accs))
            
            #
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            #

            x, y = forget_set.tensors

            for i in range(0, x.shape[0], BATCH_SIZE):

                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

            forget_accs.append(np.mean(accs))
        
        # save
        torch.save(M_unlearned.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))

    return train_times, train_accs, test_accs, forget_accs

In [15]:
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/finetune.py#L49
LR = 0.1

In [16]:
results = list()

for percentage in tqdm(PERCENTAGES):
    
    model = CNN().cuda()
    
    model.load_state_dict(torch.load('./weights/original/005.pt'))
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/unrolling-sgd/finetune/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

In [17]:
results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv('results/unrolling-sgd/finetune.csv')

results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1        0.093071   0.098697  0.097943    0.097892
           2        0.142839   0.098697  0.097943    0.097892
           3        0.106299   0.098697  0.097943    0.097892
           4        0.107764   0.098697  0.097943    0.097892
           5        0.107549   0.098697  0.097943    0.097892
10         1        1.582503   0.098711  0.097943    0.098116
           2        1.691955   0.098711  0.097943    0.098116
           3        1.651448   0.098711  0.097943    0.098116
           4        1.670228   0.098711  0.097943    0.098116
           5        1.899124   0.098711  0.097943    0.098116
20         1        3.626478   0.098271  0.097943    0.099337
           2        3.527236   0.098271  0.097943    0.099337
           3        3.505100   0.098271  0.097943    0.099337
           4        3.649031   0.098271  0.097943    0.099337
           5        3.531990   0.098271  0.097943    0.099337
30         1        5.671814   0.098558  0.097943    0.098637
           2        5.671598   0.098558  0.097943    0.098637
           3        5.404320   0.098558  0.097943    0.098637
           4        5.204876   0.098558  0.097943    0.098637
           5        5.338945   0.098558  0.097943    0.098637
40         1        7.446170   0.098389  0.097943    0.098836
           2        7.079574   0.098389  0.097943    0.098836
           3        7.060263   0.098389  0.097943    0.098836
           4        7.006691   0.098389  0.097943    0.098836
           5        6.947205   0.098389  0.097943    0.098836
50         1        9.374218   0.098714  0.097943    0.098521
           2        8.908031   0.098714  0.097943    0.098521
           3        9.111207   0.098714  0.097943    0.098521
           4        8.902072   0.098714  0.097943    0.098521
           5        9.005178   0.098714  0.097943    0.098521
60         1       11.377407   0.099542  0.097943    0.098118
           2       10.725685   0.099542  0.097943    0.098118
           3       10.559859   0.099542  0.097943    0.098118
           4       10.709129   0.099542  0.097943    0.098118
           5       10.657964   0.099542  0.097943    0.098118
70         1       13.166406   0.098857  0.097943    0.098536
           2       12.421486   0.098857  0.097943    0.098536
           3       12.464874   0.098857  0.097943    0.098536
           4       12.200490   0.098857  0.097943    0.098536
           5       12.027715   0.098857  0.097943    0.098536
80         1       15.167992   0.099500  0.097943    0.098421
           2       14.071881   0.099500  0.097943    0.098421
           3       14.064431   0.099500  0.097943    0.098421
           4       14.120578   0.099500  0.097943    0.098421
           5       14.100672   0.099500  0.097943    0.098421
90         1       16.934073   0.097739  0.097943    0.098669
           2       15.744517   0.097739  0.097943    0.098669
           3       15.703261   0.097739  0.097943    0.098669
           4       16.000236   0.097739  0.097943    0.098669
           5       16.107888   0.097739  0.097943    0.098669
99         1       19.186974   0.095943  0.097943    0.098589
           2       17.958012   0.095943  0.097943    0.098589
           3       17.719515   0.095943  0.097943    0.098589
           4       18.192384   0.095943  0.097943    0.098589
           5       17.826000   0.095943  0.097943    0.098589

# correlation

In [9]:
def std_loss(x, y):
    log_prob = -1.0 * F.log_softmax(x, 1)
    loss = log_prob.gather(1, y.unsqueeze(1))
    loss = loss.mean()
    avg_std = torch.sum(torch.std(x, dim=1))/(len(x.view(-1)))
    loss = loss + STD_REG*avg_std
    return loss

def my_cross_entropy(x, y):
    log_prob = -1.0 * F.log_softmax(x, 1)
    loss = log_prob.gather(1, y.unsqueeze(1))
    loss = loss.mean()

    #x is (N,C)
    N,C = x.shape
    p = F.softmax(x,1)
    hessian_loss = torch.sum(p *(1-p),dim =1)
    hessian_loss = hessian_loss.mean()

    loss = loss + STD_REG * hessian_loss
    return loss

# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L130

loss_fn = torch.nn.CrossEntropyLoss()

In [10]:
def fit(model, save_dir, train_set, test_set, forget_set):
    
    os.makedirs(save_dir, exist_ok=True)
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L161

    M = copy.deepcopy(model)

    M.train()
    
    # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L172
    optimizer = torch.optim.SGD(M.parameters(), lr=LR, weight_decay = L2)
    
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    forget_loader = torch.utils.data.DataLoader(forget_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    train_times = list()
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):
        
        # train
        
        train_time = 0
        
        start_time = time.time()
        
        # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L200
        
        # only update M:
        
        M.train()
        
        for x, y in forget_loader:
        
            optimizer.zero_grad()
            
            outputs_M = M(x.cuda())
            
            targets = y.cuda()
        
            if LOSS_FUNC == 'hess':
                loss_M = my_cross_entropy(outputs_M, targets)
            if LOSS_FUNC =='std':
                loss_M = std_loss(outputs_M, targets)
            if LOSS_FUNC =='regular':
                loss_M = loss_fn(outputs_M, targets)
                
            loss_M.backward()
            optimizer.step()
        
        # now we also want to compute the gradient:
        
        grad_list = []
        
        for x, y in train_loader:

            output_grad = M(x.cuda())
            
            label = y.cuda()
            
            if LOSS_FUNC == 'hess':
                loss_grad = my_cross_entropy(output_grad, label)
            if LOSS_FUNC =='std':
                loss_grad = std_loss(output_grad, label)
            if LOSS_FUNC =='regular':
                loss_grad = loss_fn(output_grad, label)
            
            loss_grad.backward(retain_graph=True)
            grads = torch.autograd.grad(loss_grad, [param for param in M.parameters()], create_graph = True)
            # !!! modified !!! original code cause OOM
            # grad_list.append(grads)
            grad_list.append([i.detach().cpu() for i in grads])
        
        # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L227
            
        # update both M and M'
        
        for x, y in forget_loader:
        
            optimizer.zero_grad()
            
            outputs_M = M(x.cuda())
            
            targets = y.cuda()
        
            if LOSS_FUNC == 'hess':
                loss_M = my_cross_entropy(outputs_M, targets)
            if LOSS_FUNC =='std':
                loss_M = std_loss(outputs_M, targets)
            if LOSS_FUNC =='regular':
                loss_M = loss_fn(outputs_M, targets)
                
            loss_M.backward()
            optimizer.step()
            
        # Now, get M''_(N+t)
            
        # https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L270
        
        M_unlearned = copy.deepcopy(M)
        
        old_params = {}
        for i, (name, params) in enumerate(M.named_parameters()):
            old_params[name] = params.clone()
            for grads in grad_list:
                old_params[name] += LR * grads[i].cuda()
        for name, params in M_unlearned.named_parameters():
            params.data.copy_(old_params[name])
        
            
        train_time += time.time() - start_time

        start_time = time.time()
            
        train_times.append(train_time)
        
        # test
            
        M_unlearned.eval()
        with torch.no_grad():
            
            #
            
            x, y = train_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            train_accs.append(np.mean(accs))
            
            #
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            #

            x, y = forget_set.tensors

            for i in range(0, x.shape[0], BATCH_SIZE):

                output = M_unlearned(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

            forget_accs.append(np.mean(accs))
        
        # save
        torch.save(M_unlearned.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))

    return train_times, train_accs, test_accs, forget_accs

In [11]:
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L167
LR = 0.01
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L72
L2 = 0.
# https://github.com/cleverhans-lab/unrolling-sgd/blob/main/Unlearning/Correlation_unlearning.py#L71
STD_REG = 0.

LOSS_FUNC = 'hess'

In [12]:
results = list()

for percentage in tqdm(PERCENTAGES):
    
    model = CNN().cuda()
    
    model.load_state_dict(torch.load('./weights/original/005.pt'))
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/unrolling-sgd/correlation/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

In [13]:
results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv('results/unrolling-sgd/correlation.csv')

results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1       23.489917   0.048499  0.048123    0.049354
           2       21.695684   0.092151  0.096146    0.096260
           3       21.668467   0.103847  0.106430    0.106771
           4       22.274964   0.099068  0.100839    0.101311
           5       21.788038   0.112177  0.113518    0.114521
10         1       21.201748   0.096694  0.098243    0.100299
           2       21.727305   0.090640  0.088658    0.087512
           3       21.535512   0.102007  0.103934    0.105851
           4       21.748307   0.112411  0.113518    0.112837
           5       21.600991   0.172912  0.173223    0.170721
20         1       21.533739   0.098417  0.097344    0.095476
           2       21.860074   0.099688  0.097843    0.095930
           3       22.119174   0.098396  0.095946    0.098610
           4       22.203216   0.141708  0.143171    0.139172
           5       22.255934   0.129937  0.133886    0.128725
30         1       22.104795   0.101128  0.101038    0.103382
           2       22.353467   0.111696  0.113518    0.113799
           3       22.309126   0.208754  0.210763    0.210866
           4       23.712210   0.101128  0.101038    0.103382
           5       22.500949   0.162676  0.167033    0.158355
40         1       22.534714   0.054222  0.054413    0.052828
           2       22.704143   0.218028  0.213958    0.211518
           3       22.819202   0.135972  0.128994    0.134819
           4       23.424061   0.197167  0.191494    0.193938
           5       23.091496   0.232028  0.232927    0.227011
50         1       23.031690   0.042044  0.039936    0.041017
           2       23.550739   0.112673  0.116813    0.116132
           3       23.665971   0.129398  0.131290    0.128947
           4       25.195868   0.102312  0.105831    0.101419
           5       24.468297   0.243104  0.252396    0.246278
60         1       23.054505   0.236250  0.248003    0.245697
           2       23.687860   0.100250  0.100040    0.098444
           3       23.533322   0.107917  0.108726    0.105463
           4       23.395105   0.263792  0.264876    0.265994
           5       23.916171   0.156417  0.165136    0.162400
70         1       23.644594   0.125111  0.121306    0.122905
           2       24.038399   0.197935  0.195887    0.192170
           3       24.608619   0.102964  0.103335    0.100669
           4       24.982249   0.342917  0.355132    0.347978
           5       23.797917   0.088921  0.091354    0.091271
80         1       24.573494   0.099833  0.097344    0.097008
           2       24.447432   0.269500  0.276058    0.271322
           3       25.575164   0.471250  0.491414    0.485814
           4       25.314490   0.162500  0.166833    0.165058
           5       25.083449   0.408917  0.407149    0.410145
90         1       25.331308   0.179189  0.193690    0.187031
           2       25.100163   0.394781  0.394469    0.391945
           3       24.448052   0.360040  0.350739    0.351512
           4       24.866441   0.396775  0.389377    0.384402
           5       26.011507   0.785239  0.788538    0.784483
99         1       26.465792   0.957237  0.965555    0.967540
           2       25.611852   0.978618  0.985923    0.986838
           3       25.715510   0.970395  0.982228    0.983813
           4       26.242393   0.976974  0.985024    0.984807
           5       25.511864   0.985197  0.986921    0.988623